In [ ]:
import json
import re
import pandas as pd 
import numpy as np
import jsonlines
import copy

In [14]:
filename=['中国图象图形学报','中文信息学报','模式识别与人工智能','计算机应用','计算机辅助设计与图形学学报']

In [15]:
DATA=[]
for f in filename:
    with open("./DATA/"+f+".jsonl","r+",encoding="utf-8") as ori:
        for item in jsonlines.Reader(ori):
            DATA.append(item)

In [16]:
print(len(DATA))
print(DATA[0])
print(DATA[0]['label'])

1501
{'id': 32764, 'text': '3D多尺度深度卷积神经网络肺结节检测', 'label': [[5, 13, 'method'], [13, 18, 'RP'], [13, 18, 'rp']], 'Comments': []}
[[5, 13, 'method'], [13, 18, 'RP'], [13, 18, 'rp']]


In [17]:
for i in range(len(DATA)):
    DATA[i]['id']=i+1

In [19]:
method = pd.read_csv('./DATA/method.csv')
method_dict={}
for i in range(len(method)):
    mi = method.iloc[i].dropna().values
    for j in range(len(mi)):
        method_dict.update({mi[j]:mi[0]})

In [20]:
problems = pd.read_csv('./DATA/problems.csv')
problems_dict={}
for i in range(len(problems)):
    pi = problems.iloc[i].dropna().values
    for j in range(len(pi)):
        problems_dict.update({pi[j]:pi[0]})      

In [21]:
data_list=[]
for i in range(len(DATA)):
    num_m=[]
    num_rp=[]
    for j in range(len(DATA[i]['label'])):
        if DATA[i]['label'][j][2]=='method':
            try:
                m_i_ = DATA[i]['text'][int(DATA[i]['label'][j][0]):int(DATA[i]['label'][j][1])]
                m_i = method_dict[m_i_]
                data_list.append([DATA[i]['id'],'paper','use','method',m_i])
                num_m.append(m_i)
            except:
                continue

            
        if DATA[i]['label'][j][2]=='rp':
            try:
                p_i_ = DATA[i]['text'][int(DATA[i]['label'][j][0]):int(DATA[i]['label'][j][1])]
                p_i = problems_dict[p_i_]
                data_list.append([DATA[i]['id'],'paper','focus','problems',p_i])
                num_rp.append(p_i)
            except:
                continue
                
        for m in num_m:
            for rp in num_rp:
                data_list.append([rp,'problems','solve','method',m])
                
#                 data_list.append([m,'method','solve','problems',rp])
        

In [22]:
columns=['head_entity', 'head_type','relation','tail_type','tail_entity']

In [23]:
connect = pd.read_csv('./connected.csv',index_col=0).reset_index(drop=True)
for i in range(len(connect)):
    connect.iloc[i][1] = connect.iloc[i][1].replace("['",'').replace("']",'')
    data_list.append([connect.iloc[i][0],'problems','connect','ontology',connect.iloc[i][1]])

In [24]:
Ontology = pd.read_csv('./Ont.csv',encoding="utf_8_sig",index_col=0,header=1).reset_index(drop=True)
for i in range(len(Ontology)):
    data_list.append([Ontology.iloc[i][0],'ontology','hyponymy','ontology',Ontology.iloc[i][1]])
len(data_list)

6261

In [25]:
df_dict = dict(zip(columns,np.array(data_list).T))
df_data = pd.DataFrame(df_dict)

In [26]:
df_data.to_csv('./neo4j.csv',encoding="utf_8_sig",index=None)

In [24]:
import copy
from py2neo import Node, Relationship, Graph, NodeMatcher, RelationshipMatcher,Subgraph

In [26]:
graph = Graph("http://localhost:7474/", auth=("neo4j", "Emily525"), name='test')

In [16]:
path = r"./neo4j.csv"
file = open(path,'r',encoding = 'utf-8').readlines()

con = list()
zh = ['paper','method','problems','ontology','use','solve','focus','hyponymy','connect']

In [17]:
for j in file:
    try:
        a = j.replace("\t","").strip('\n').split(",")
        # a = [eval(i) for i in a] #引号里面有引号-两对引号
        a = [i for i in a] #只有一对引号
        if (a[1] not in zh) or (a[2] not in zh) or (a[3] not in zh):
            print(a)
            continue
        con.append(a)
    except:
        continue

['\ufeffhead_entity', 'head_type', 'relation', 'tail_type', 'tail_entity']


In [28]:
for j in con_:
    selector = NodeMatcher(graph) #创建图，实质上为句柄
        # ---创建头实体节点---
    entity1 = selector.match(j[1], name = j[0]) #检索是否存在头实体节点
    if len(list(entity1)) == 0: #不存在头实体节点，则创建头实体
        entity1_node = Node(j[1], name = j[0])
        graph.create(entity1_node) #创建头实体
    else: #存在头实体节点，跳过
        pass

        # ---创建尾实体节点---
    entity3 = selector.match(j[3], name = j[4])
    if len(list(entity3)) == 0: #不存在尾实体节点，则创建
        
        entity3_node = Node(j[3], name = j[4])
        graph.create(entity3_node)   
    else: #存在节点，跳过
        pass

In [29]:
for j in con_:
        #将两虚点建立关系
    e1_node = graph.nodes.match(j[1], name = j[0]).first()
    e2_node = graph.nodes.match(j[3], name = j[4]).first()
    e12 = Relationship(e1_node, j[2], e2_node)
    graph.create(e12)

print("ok1")

ok1
